#### Sunday, December 17, 2023

[Tutorial: Fine-Tuning a Model on Your Own Data](https://haystack.deepset.ai/tutorials/02_finetune_a_model_on_your_data)

This notebook errors when trying to load the model it trained from the mymodel local folder using FARMReader ... sigh.

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
!ls /root/.cache/huggingface/hub

models--HuggingFaceH4--zephyr-7b-beta
models--bert-base-uncased
models--deepset--roberta-base-squad2
models--distilbert-base-uncased-distilled-squad
models--google--flan-t5-large
models--meta-llama--Llama-2-13b-hf
models--my_model--language_model.bin
models--sentence-transformers--all-MiniLM-L6-v2
models--sentence-transformers--multi-qa-mpnet-base-dot-v1
version.txt


In [3]:
from huggingface_hub import login
login()

# Tutorial: Fine-Tuning a Model on Your Own Data

- **Level**: Intermediate
- **Time to complete**: 15 minutes
- **Nodes Used**: `FARMReader`
- **Goal**: After completing this tutorial, you will have learned how to fine-tune a pretrained Reader model on your own data.

## Overview

For many use cases it is sufficient to just use one of the existing public models that were trained on SQuAD or other public QA datasets (e.g. Natural Questions).
However, if you have domain-specific questions, fine-tuning your model on custom examples will very likely boost your performance.
While this varies by domain, we saw that ~ 2000 examples can easily increase performance by +5-20%.



## Preparing the Colab Environment

- [Enable GPU Runtime in Colab](https://docs.haystack.deepset.ai/docs/enabling-gpu-acceleration#enabling-the-gpu-in-colab)
- [Set logging level to INFO](https://docs.haystack.deepset.ai/docs/log-level)

## Installing Haystack

To start, let's install the latest release of Haystack with `pip`:

In [ ]:
# %%bash

# pip install --upgrade pip
# pip install farm-haystack[colab,inference]

### Enabling Telemetry 
Knowing you're using this tutorial helps us decide where to invest our efforts to build a better product but you can always opt out by commenting the following line. See [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry) for more details.

In [4]:
from haystack.telemetry import tutorial_running

tutorial_running(2)


## Create Training Data

There are two ways to generate training data:

1. **Annotation**: You can use the [annotation tool](https://haystack.deepset.ai/guides/annotation) to label your data, which means highlighting answers to your questions in a document. The tool supports structuring your workflow with organizations, projects, and users. The labels can be exported in SQuAD format that is compatible for training with Haystack.

![Snapshot of the annotation tool](https://raw.githubusercontent.com/deepset-ai/haystack/main/docs/img/annotation_tool.png)

2. **Feedback**: For production systems, you can collect training data from direct user feedback via Haystack's [REST API interface](https://github.com/deepset-ai/haystack#rest-api). This includes a customizable user feedback API for providing feedback on the answer returned by the API. The API provides a feedback export endpoint to obtain the feedback data for fine-tuning your model further.


## Fine-Tune Your Model

Once you have collected training data, you can fine-tune your base model. To do that, you need to initialize a reader as a base model and fine-tune it on your custom dataset (should be in SQuAD-like format). We recommend using a base model that was trained on SQuAD or a similar QA dataset beforehand to benefit from Transfer Learning effects.

**Recommendation**: Run training on a GPU.
If you are using Colab: Enable this in the menu "Runtime" > "Change Runtime type" > Select "GPU" in dropdown.
Then change the `use_gpu` arguments below to `True`

1. Initialize a `Reader` with the model to fine-tune:

In [5]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="distilbert-base-uncased-distilled-squad", use_gpu=True)

# 3m 47.0s

2. Get SQUAD-style data for training. You can use this dataset we prepared:

In [6]:
from haystack.utils import fetch_archive_from_http

data_dir = "data/fine-tuning"


fetch_archive_from_http(
    url="https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-downstream/squad20.tar.gz", output_dir=data_dir
)

# 9.1s

False

3. Train the model on your own data and save it to "my_model"

In [7]:
reader.train(data_dir=data_dir, train_filename="squad20/dev-v2.0.json", use_gpu=True, n_epochs=1, save_dir="my_model")

#46.5s

Preprocessing dataset:   0%|          | 0/3 [00:00<?, ? Dicts/s]

Train epoch 0/0 (Cur. train loss: 1.1044): 100%|██████████| 1360/1360 [00:42<00:00, 31.91it/s]


4. Initialize a new reader with your fine-tuned model:

In [8]:
new_reader = FARMReader(model_name_or_path="my_model", use_gpu=True, local_files_only=True, force_download=False)

OSError: my_model/language_model.bin is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

5. Finally, use the `new_reader` that was initialized with your fine-tuned model.

In [ ]:
from haystack.schema import Document

new_reader.predict(
    query="What is the capital of Germany?", documents=[Document(content="The capital of Germany is Berlin")]
)

Congratulations! 🎉 You’ve fine-tuned a base model on your own data!